In [ ]:
import sys
import re
import itertools
import pathlib
import numpy
import scipy.interpolate
import ipywidgets
import pandas
from h5py import File as h5open
from cycler import cycler
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import ticker
from matplotlib.legend_handler import HandlerTuple

In [ ]:
# find helpers and locate workdir
for parent in [pathlib.Path.cwd()] + list(pathlib.Path.cwd().parents):
    if parent.joinpath("modulus").is_dir():
        projdir = parent
        sys.path.insert(0, str(projdir.joinpath("modulus")))
        from helpers.utils import read_tensorboard_data  # pylint: disable=import-error
        from helpers.lr_simulator import widget as lr_widget
        from helpers.utils import log_parser  # pylint: disable=import-error
        break
else:
    raise FileNotFoundError("Couldn't find module `helpers`.")

# point workdir to the correct folder
workdir = projdir.joinpath("modulus", "tgv-2d-re100")
petibmdir = projdir.joinpath("petibm", "taylor-green-vortex-2d-re100")
figdir = workdir.joinpath("figures")
figdir.mkdir(exist_ok=True)

In [ ]:
# unified figure style
pyplot.style.use(projdir.joinpath("resources", "figstyle"))

In [ ]:
# parameters
nls = [1, 2, 3]
nns = [16, 32, 64, 128, 256]
nbss = [1024, 2048, 4096, 8192, 16384, 32768, 65536]

In [ ]:
def get_best_med_worst_base_cases():
    """Get the cases with the best, median, and the worst errors.
    """
    bestl2norm = float("inf")
    best = None
    errs = []
    cases = []
    for nl, nn, nbs in itertools.product(nls, nns, nbss):
        with h5open(workdir.joinpath("outputs", "base-cases", f"nl{nl}-nn{nn}-npts{nbs}-raw.h5"), "r") as h5file:
            errs.append(float(h5file[f"sterrs/u/l2norm"][...]))
        cases.append((nl, nn, nbs))
    
    errs = numpy.array(errs)
    cases = numpy.array(cases, dtype=object)

    besterr = numpy.min(errs)
    bestconf = cases[numpy.argmin(errs)]
    worsterr = numpy.max(errs)
    worstconf = cases[numpy.argmax(errs)]
    mederr = numpy.median(errs)
    medconf = cases[numpy.where(errs == mederr)[0][0]]
    
    return (besterr, bestconf), (mederr, medconf), (worsterr, worstconf)

In [ ]:
(besterr, bestconf), (mederr, medconf), (worsterr, worstconf) = get_best_med_worst_base_cases()
print(f"raw, (best case, err): {besterr}, {bestconf}")
print(f"raw, (median case, err): {mederr}, {medconf}")
print(f"raw, (worst case, err): {worsterr}, {worstconf}")

In [ ]:
def plot_base_case_training_history(workdir, h5dir, h5dir2, figdir, nl, nn, nbs, ws, close=False):
    """Plot figures related to training loss and spatial-temporal errors.
    """

    data = log_parser(workdir.joinpath(f"nl{nl}-nn{nn}-npts{nbs}"))

    with h5open(h5dir.joinpath(f"nl{nl}-nn{nn}-npts{nbs}-raw.h5"), "r") as h5file:
        errsteps = h5file["walltime/steps"][...]
        errtimes = h5file["walltime/elapsedtimes"][...]
        err = [
            h5file["walltime/l2norms/u/0.0"][...],
            h5file["walltime/l2norms/u/40.0"][...],
            h5file["walltime/l2norms/u/80.0"][...],
        ]

    with h5open(h5dir2.joinpath(f"nl{nl}-nn{nn}-npts{nbs}-raw.h5"), "r") as h5file:
        sterr = h5file["walltime/l2norms/u"][...]

    # plot according to optimizer type
    fig, ax = pyplot.subplots(1, 1, sharex=False, sharey=False, figsize=(8, 4))
    fig.suptitle(rf"2D TGV, $Re=100$, $(N_l, N_n, N_{{bs}})=({nl}, {nn}, {nbs})$")
    
    # loss against steps
    ax.set_title("Training loss and solution errors v.s. iterations")
    l1, = ax.semilogy(data.index, data.loss.rolling(window=ws).min(), lw=2, label="Aggregated loss")
    l2, = ax.semilogy(errsteps, sterr, lw=2, ls="--", label=r"Overall spatial-temporal error")

    l4, = ax.semilogy(errsteps, err[0], lw=1.5, alpha=0.8, ls="--", label=r"$t=0$")
    l5, = ax.semilogy(errsteps, err[1], lw=1.5, alpha=0.8, ls="--", label=r"$t=40$")
    l6, = ax.semilogy(errsteps, err[2], lw=1.5, alpha=0.8, ls="--", label=r"$t=80$")

    # customized  legend locations
    if (nl, nn, nbs) == (2, 32, 16384):
        lloc1 = (0.55, 0.6)
        lloc2 = (0.99, 0.6)
    if (nl, nn, nbs) == (2, 32, 65536):
        lloc1 = (0.55, 0.6)
        lloc2 = (0.99, 0.6)
    else:
        lloc1 = (0.55, 0.99)
        lloc2 = (0.99, 0.99)
    
    # legends
    lgd1 = ax.legend(handles=[l1, l2], loc="upper right", bbox_to_anchor=lloc1)
    lgd2 = ax.legend(
        handles=[l4, l5, l6], title=r"Spatial error of $u$",
        loc="upper right", bbox_to_anchor=lloc2, ncol=3
    )
    ax.add_artist(lgd1)

    ax.set_xlabel("Iteration")
    ax.set_ylabel(r"Loss or $L_2$ error")
    ax.grid()

    # time axis / errors
    axerr = ax.twiny()
    axerr.semilogy(errtimes, err[0], lw=0)  # dummy line to set x-axis limits
    axerr.spines["bottom"].set_position(("axes", -0.3))
    axerr.spines["bottom"].set_visible(True)
    axerr.xaxis.set_label_position("bottom")
    axerr.xaxis.set_ticks_position("bottom")
    axerr.set_xlabel("Run time (hours)")
    axerr.get_xaxis().set_ticks_position("bottom")

    # save
    figdir.joinpath("training-hist").mkdir(parents=True, exist_ok=True)
    fig.savefig(figdir.joinpath("training-hist", f"nl{nl}-nn{nn}-npts{nbs}.png"))

    if close:
        pyplot.close(fig)

In [ ]:
# for nl, nn, nbs in itertools.product(nls, nns, nbss):
#     print(nl, nn, nbs)
#     plot_base_case_training_history(
#         workdir=workdir.joinpath("base-cases"),
#         h5dir=workdir.joinpath("outputs", "base-cases"),
#         h5dir2=workdir.joinpath("old.nogit", "outputs.sterrs", "base-cases"),
#         figdir=figdir.joinpath("base-cases"),
#         nl=nl, nn=nn, nbs=nbs, ws=10, close=True
#     )

In [ ]:
# plot case training history
option1 = ipywidgets.Dropdown(options=nls, value=3)
option2 = ipywidgets.Dropdown(options=nns, value=256)
option3 = ipywidgets.Dropdown(options=nbss, value=8192)
option4 = ipywidgets.IntSlider(value=10, min=1, max=200, step=2, orientation="horizontal") 
canvas = ipywidgets.interactive_output(
    plot_base_case_training_history,
    {
        "workdir": ipywidgets.fixed(workdir.joinpath("base-cases")),
        "h5dir": ipywidgets.fixed(workdir.joinpath("outputs", "base-cases")),
        "h5dir2": ipywidgets.fixed(workdir.joinpath("old.nogit", "outputs.sterrs", "base-cases")),
        "figdir": ipywidgets.fixed(figdir.joinpath("base-cases")),
        "nl": option1, "nn": option2, "nbs": option3, "ws": option4,
    }
)

out = ipywidgets.VBox([option1, option2, option3, option4, canvas])
display(out)

In [ ]:
def plot_base_case_contour(nl, nn, nbs, time, workdir, figdir, close=False):
    """Plot figures for the case performed the best at t=40.
    """

    with h5open(workdir.joinpath(f"nl{nl}-nn{nn}-npts{nbs}-raw.h5"), "r") as h5file:
        coords = (h5file["field/x"][...], h5file["field/y"][...])

        vals = {
            r"$u$": h5file[f"field/{time}/u"][...],
            r"$v$": h5file[f"field/{time}/v"][...],
            r"$p$": h5file[f"field/{time}/p"][...],
            r"$\omega_z$": h5file[f"field/{time}/vorticity_z"][...],
        }

        errs = {
            r"$u$": h5file[f"field/{time}/err-u"][...],
            r"$v$": h5file[f"field/{time}/err-v"][...],
            r"$p$": h5file[f"field/{time}/err-p"][...],
            r"$\omega_z$": h5file[f"field/{time}/err-vorticity_z"][...],
        }
    
    # use analytical solutions for levels
    lvls = {  # calculate solutions firstj
        r"$u$": numpy.cos(coords[0]) * numpy.sin(coords[1]) * numpy.exp(-2.*0.01*float(time)),
        r"$v$": - numpy.sin(coords[0]) * numpy.cos(coords[1]) * numpy.exp(-2.*0.01*float(time)),
        r"$p$": - numpy.exp(-4.*0.01*float(time)) * (numpy.cos(2.*coords[0]) + numpy.cos(2.*coords[1])) / 4.,
        r"$\omega_z$": - 2. * numpy.cos(coords[0]) * numpy.cos(coords[1]) * numpy.exp(-2.*0.01*float(time)),
    }

    # actually convert them to levels
    lvls = {field: numpy.linspace(val.min(), val.max(), 17) for field, val in lvls.items()}

    # normalizer for error contourf
    normerr = colors.LogNorm

    # re-cal. the pressure w/ the mean from analytical soln. as it is assumed to have a constant shift
    ptrue = - numpy.exp(-4.*0.01*float(time)) * (numpy.cos(2.*coords[0]) + numpy.cos(2.*coords[1])) / 4.
    vals[r"$p$"] = vals[r"$p$"] - vals[r"$p$"].mean()
    errs[r"$p$"] = abs(vals[r"$p$"] - ptrue)

    fig, axs = pyplot.subplots(4, 2, sharex=True, sharey=True, figsize=(8.5, 13))

    fig.suptitle(
        rf"Flow and errors, TGV 2D@$t={float(time)}$, $Re=100$, "+"\n" +
        rf"$(N_l, N_n, N_{{bs}})=({nl}, {nn}, {nbs})$"
    )

    for i, ((field, err), val, lvl) in enumerate(zip(errs.items(), vals.values(), lvls.values())):
        # field values
        ct = axs[i, 0].contourf(*coords, val, lvl, extend="both")
        axs[i, 0].set_aspect("equal")
        axs[i, 0].set_title(field)
        fig.colorbar(ct, ax=axs[i, 0], extend="both")

        # errors
        ct = axs[i, 1].contourf(*coords, err, 17, extend="both", norm=normerr(err.min(), err.max()))
        axs[i, 1].set_aspect("equal")
        axs[i, 1].set_title(f"Absolute error, {field}")
        fmt = ticker.LogFormatter()
        cbar = fig.colorbar(ct, ax=axs[i, 1], format=fmt, extend="both")
    
    axs[0, 0].set_ylabel(r"$y$")
    axs[1, 0].set_ylabel(r"$y$")
    axs[2, 0].set_ylabel(r"$y$")
    axs[3, 0].set_ylabel(r"$y$")
    axs[3, 0].set_xlabel(r"$x$")
    axs[3, 1].set_xlabel(r"$x$")

    figdir.joinpath("contours").mkdir(parents=True, exist_ok=True)
    pyplot.savefig(figdir.joinpath("contours", f"nl{nl}-nn{nn}-npts{nbs}-t{time}.png"))

    if close:
        pyplot.close(fig)

In [ ]:
# for nl, nn, nbs in itertools.product(nls, nns, nbss):
#     plot_base_case_contour(
#         nl=nl, nn=nn, nbs=nbs, time="40.0",
#         workdir=workdir.joinpath("outputs", "base-cases"),
#         figdir=figdir.joinpath("base-cases"), close=True
#     )

In [ ]:
# plot contours for cases
option1 = ipywidgets.Dropdown(options=nls, value=2)
option2 = ipywidgets.Dropdown(options=nns, value=32)
option3 = ipywidgets.Dropdown(options=nbss, value=65536)
option4 = ipywidgets.Dropdown(options=["0.0", "40.0", "80.0"], value="40.0")
canvas = ipywidgets.interactive_output(
    plot_base_case_contour,
    {
        "nl": option1, "nn": option2, "nbs": option3, "time": option4,
        "workdir": ipywidgets.fixed(workdir.joinpath("outputs", "base-cases")),
        "figdir": ipywidgets.fixed(figdir.joinpath("base-cases"))
    }
)
display(ipywidgets.VBox([option1, option2, option3, option4, canvas]))

In [ ]:
def plot_err_arch_boxplot(field, workdir, figdir):
    """plot_err_arch_boxplot
    """
    data = {"nl": [], "nn": [], "nbs": [], "l2norm": []}
    for nl, nn, nbs in itertools.product(nls, nns, nbss):
        data["nl"].append(nl)
        data["nn"].append(nn)
        data["nbs"].append(nbs)
        with h5open(workdir.joinpath(f"nl{nl}-nn{nn}-npts{nbs}-raw.h5"), "r") as h5file:
            data["l2norm"].append(float(h5file[f"sterrs/{field}/l2norm"][...]))

    data = pandas.DataFrame(data)
    data = data.pivot(index="nbs", columns=["nl", "nn"], values="l2norm")
    data = data[data.mean().sort_values(ascending=False).index]

    fig, ax = pyplot.subplots(1, 1)
    fig.suptitle(r"Error distribution across network architectures, " rf"${field}$")

    ax.boxplot(
        data.values, labels=data.columns, showmeans=True,
        meanprops={"marker": ".", "mfc": "k", "mec": "k"},
        medianprops={"ls": "none"},
    )
    ax.tick_params(axis="x", labelrotation=45)
    ax.set_xlabel(r"$(N_l, N_n)$")
    ax.set_ylabel(rf"$L_2$ error of ${field}$")
    ax.set_yscale("log")

    figdir.joinpath("err-vs-arch").mkdir(parents=True, exist_ok=True)
    fig.savefig(figdir.joinpath("err-vs-arch", f"err-arch-boxplot-{field}.png"))

plot_err_arch_boxplot("u", workdir.joinpath("outputs", "base-cases"), figdir.joinpath("base-cases"))
plot_err_arch_boxplot("v", workdir.joinpath("outputs", "base-cases"), figdir.joinpath("base-cases"))

In [ ]:
def dof_calculator(nl, nn, dim=2, unsteady=True, periodic=True):
    """Degree of freedom calculator.
    """
    ninp = dim + int(unsteady) + 2 * int(periodic)
    dof = ninp * nn + 2 * nn
    dof += (nn**2 + 2 * nn) * (nl - 1)
    dof += (nn + 1) * (dim + 1) 
    return dof

In [ ]:
def plot_dof_err(field, workdir, figdir):
    """plot_dof_err
    """
    data = {"dofs": [], "l2norm": [], "nbs": []}
    for nl, nn, nbs in itertools.product(nls, nns, nbss):
        data["nbs"].append(nbs)
        data["dofs"].append(dof_calculator(nl, nn, 2, True, True))
        with h5open(workdir.joinpath(f"nl{nl}-nn{nn}-npts{nbs}-raw.h5"), "r") as h5file:
            data["l2norm"].append(float(h5file[f"sterrs/{field}/l2norm"][...]))

    data = pandas.DataFrame(data)
    data = data.pivot(index="nbs", columns=["dofs"], values="l2norm")
    data = data.sort_index(axis=1)

    # box widths on the plot with log x axis
    width = lambda p, w: 10**(numpy.log10(p)+w/2.)-10**(numpy.log10(p)-w/2.)

    fig, ax = pyplot.subplots(1, 1)
    fig.suptitle(r"Error distribution v.s. degree of freedom, " rf"${field}$")

    ax.boxplot(
        data.values, labels=data.columns, positions=data.columns,
        showmeans=True, widths=width(data.columns, 0.1),
        meanprops={"marker": ".", "mfc": "k", "mec": "k"},
        medianprops={"ls": "none"},
    )

    ax.set_xlabel("Degree of freedom")
    ax.set_xscale("log")

    ax.set_ylabel(rf"$L_2$ error of ${field}$")
    ax.set_yscale("log")

    figdir.joinpath("err-vs-arch").mkdir(parents=True, exist_ok=True)
    fig.savefig(figdir.joinpath("err-vs-arch", f"err-dof-boxplot-{field}.png"))

plot_dof_err("u", workdir.joinpath("outputs", "base-cases"), figdir.joinpath("base-cases"))
plot_dof_err("v", workdir.joinpath("outputs", "base-cases"), figdir.joinpath("base-cases"))

In [ ]:
# display(lr_widget())

In [ ]:
def plot_error_loss_scatter(workdir, outdir, figdir, field):
    """Plot error vs loss.
    """

    loss = []
    errs = []
    ccodes = []
    cmap = {(nl, nn): i for i, (nl, nn) in enumerate(itertools.product(nls, nns))}
    invcmap = {i: (nl, nn) for i, (nl, nn) in enumerate(itertools.product(nls, nns))}

    for (nl, nn, nbs) in itertools.product(nls, nns, nbss):
        loss.append(log_parser(workdir.joinpath(f"nl{nl}-nn{nn}-npts{nbs}")).iloc[-1]["loss"])

        with h5open(outdir.joinpath(f"nl{nl}-nn{nn}-npts{nbs}-raw.h5"), "r") as h5file:
            errs.append(float(h5file[f"sterrs/{field}/l2norm"][...]))
        
        ccodes.append(cmap[(nl, nn)])
    
    fig, ax = pyplot.subplots(1, 1)
    fig.suptitle(rf"Error v.s. aggregated loss, ${field}$")

    scatter = ax.scatter(loss, errs, c=ccodes, s=75, cmap="tab20", alpha=0.6, marker="o")

    ax.set_xlabel("Aggregate loss")
    ax.set_xscale("log")

    ax.set_ylabel(rf"$L_2$ error of ${field}$")
    ax.set_yscale("log")

    # legend
    handles, labels = scatter.legend_elements(prop="colors", num=None, fmt="{x:d}", func=lambda x: x.astype(int))
    labels = [invcmap[int(key)] for key in labels]
    ax.legend(handles, labels, loc=0, title=r"$(N_l, N_n)$", ncol=2)

plot_error_loss_scatter(
    workdir.joinpath("base-cases"), workdir.joinpath("outputs", "base-cases"),
    workdir.joinpath("figures"), "u"
)

plot_error_loss_scatter(
    workdir.joinpath("base-cases"), workdir.joinpath("outputs", "base-cases"),
    workdir.joinpath("figures"), "v"
)

In [ ]:
# scalability for nl3-nn128-npts8192
gpus = [1, 2, 4, 8]
logdata = []
h5data = []
compdata = []
comph5data = []

for ngpus in gpus:

    bname = "exp-sum-scaling"
    cname = f"nl3-nn128-npts8192-ngpus{ngpus}"
    logdata.append(log_parser(workdir.joinpath(bname, cname)))

    with h5open(workdir.joinpath("outputs", bname, f"{cname}-raw.h5"), "r") as h5file:
        h5data.append({
            "steps": h5file["walltime/steps"][...],
            "elapsedtimes": h5file["walltime/elapsedtimes"][...],
            "l2norms": h5file["walltime/l2norms/u/40.0"][...]
        })

    bname = "base-cases"
    cname = f"nl3-nn128-npts{8192*ngpus}"
    compdata.append(log_parser(workdir.joinpath(bname, cname)))

    with h5open(workdir.joinpath("outputs", bname, f"{cname}-raw.h5"), "r") as h5file:
        comph5data.append({
            "steps": h5file["walltime/steps"][...],
            "elapsedtimes": h5file["walltime/elapsedtimes"][...],
            "l2norms": h5file["walltime/l2norms/u/40.0"][...]
        })

styles_def = (
    cycler("ls", ["solid", "dotted", "dashed", "dashdot"]*5) + 
    cycler("color", pyplot.get_cmap("tab10").colors*2) + 
    cycler("lw", [2.0]*20) +
    cycler("alpha", [0.75]*20)
)

styles = styles_def()

# ===============================================================
fig, ax = pyplot.subplots(1, 1, figsize=(8, 4))
fig.suptitle("Aggregated loss and elapsed time v.s. iteration")

ax.set_xlabel("Iteration")
ax.set_ylabel(r"Aggregated loss and $L_2$ error of $u$")
ax.grid()
tax = ax.twinx()
tax.set_ylabel("Run time (hours)")

l1s, l2s, l3s = [], [], []
for ilogdata, ih5data in zip(logdata, h5data):
    l1s.append(ax.semilogy(ilogdata.index, ilogdata.loss.rolling(window=30).min(), **next(styles))[0])
    l2s.append(ax.semilogy(ih5data["steps"], ih5data["l2norms"], **next(styles))[0])
    l3s.append(tax.plot(ilogdata.index, ilogdata["time elapsed"], **next(styles))[0])

labels = [f"{ngpus} GPUs" for ngpus in gpus]
lgds = [
    ax.legend(l1s, labels, title="Loss", loc="upper left", bbox_to_anchor=(1.07, 1.025)),
    ax.legend(l2s, labels, title=r"$L_2$ err. of $u$", loc="center left", bbox_to_anchor=(1.07, 0.5)),
    ax.legend(l3s, labels, title="Run time", loc="lower left", bbox_to_anchor=(1.07, -0.025))
]
ax.add_artist(lgds[0])
ax.add_artist(lgds[1])
ax.add_artist(lgds[2])

# save
figdir.joinpath("scaling-tests").mkdir(parents=True, exist_ok=True)
fig.savefig(figdir.joinpath("scaling-tests", "nl3-nn128-npts8192-scaling.png"), bbox_inches="tight")

# ===============================================================
fig = pyplot.figure(figsize=(8, 6.5))
fig.suptitle(r"Comparing $(N_l, N_n, N_{bs}) \times N_{gpu}$ and $(N_l, N_n, N_{bs} \times N_{gpu})$")
gs = fig.add_gridspec(2, 4)

locs = [(0, slice(None, 2)), (0, slice(2, None)), (1, slice(None, 2))]
for iloc, ngpus, ilogdata, icompdata, ih5data, icomph5data in zip(locs, gpus[1:], logdata[1:], compdata[1:], h5data[1:], comph5data[1:]):
    styles = styles_def()
    ax = fig.add_subplot(gs[iloc])
    ax.set_title(rf"$(3, 128, 8192) \times {ngpus}$ v.s. $(3, 128, {8192*ngpus})$")
    ax.set_xlabel("Iteration")
    ax.set_ylabel(r"Aggregated loss and $L_2$ error of $u$")
    tax = ax.twinx()
    tax.set_ylabel("Run time (hours)")
    l1, = ax.semilogy(ilogdata.index, ilogdata.loss.rolling(window=30).min(), **next(styles))
    l2, = ax.semilogy(icompdata.index, icompdata.loss.rolling(window=30).min(), **next(styles))
    l3, = ax.semilogy(ih5data["steps"], ih5data["l2norms"], **next(styles))
    l4, = ax.semilogy(icomph5data["steps"], icomph5data["l2norms"], **next(styles))
    l5, = tax.plot(ilogdata.index, ilogdata["time elapsed"], **next(styles))
    l6, = tax.plot(icompdata.index, icompdata["time elapsed"], **next(styles))

# legend
labels = [r"$(N_l, N_n, N_{bs})\times N_{gpu}$", r"$(N_l, N_n, N_{bs}\times N_{gpu})$"]
fig.legend([l1, l2], labels, title="Loss", loc="upper left", bbox_to_anchor=(0.6, 0.44), labelspacing=0.)
fig.legend([l3, l4], labels, title=r"$L_2$ err. of $u$", loc="upper left", bbox_to_anchor=(0.6, 0.33), labelspacing=0.)
fig.legend([l5, l5], labels, title="Run time", loc="upper left", bbox_to_anchor=(0.6, 0.213), labelspacing=0.)

# save
figdir.joinpath("scaling-tests").mkdir(parents=True, exist_ok=True)
fig.savefig(figdir.joinpath("scaling-tests", "nl3-nn128-npts8192-multi-singl-gpus.png"))

In [ ]:
# scalability for nl2-nn32-npts8192
gpus = [1, 2, 4, 8]
logdata = []
h5data = []
compdata = []
comph5data = []

for ngpus in gpus:

    bname = "exp-sum-scaling"
    cname = f"nl2-nn32-npts8192-ngpus{ngpus}"
    logdata.append(log_parser(workdir.joinpath(bname, cname)))

    with h5open(workdir.joinpath("outputs", bname, f"{cname}-raw.h5"), "r") as h5file:
        h5data.append({
            "steps": h5file["walltime/steps"][...],
            "elapsedtimes": h5file["walltime/elapsedtimes"][...],
            "l2norms": h5file["walltime/l2norms/u/40.0"][...]
        })

    bname = "base-cases"
    cname = f"nl2-nn32-npts{8192*ngpus}"
    compdata.append(log_parser(workdir.joinpath(bname, cname)))

    with h5open(workdir.joinpath("outputs", bname, f"{cname}-raw.h5"), "r") as h5file:
        comph5data.append({
            "steps": h5file["walltime/steps"][...],
            "elapsedtimes": h5file["walltime/elapsedtimes"][...],
            "l2norms": h5file["walltime/l2norms/u/40.0"][...]
        })

styles_def = (
    cycler("ls", ["solid", "dotted", "dashed", "dashdot"]*5) + 
    cycler("color", pyplot.get_cmap("tab10").colors*2) + 
    cycler("lw", [2.0]*20) +
    cycler("alpha", [0.75]*20)
)

styles = styles_def()

# ===============================================================
fig, ax = pyplot.subplots(1, 1, figsize=(8, 4))
fig.suptitle("Aggregated loss and elapsed time v.s. iteration")

ax.set_xlabel("Iteration")
ax.set_ylabel(r"Aggregated loss and $L_2$ error of $u$")
ax.grid()
tax = ax.twinx()
tax.set_ylabel("Run time (hours)")

l1s, l2s, l3s = [], [], []
for ilogdata, ih5data in zip(logdata, h5data):
    l1s.append(ax.semilogy(ilogdata.index, ilogdata.loss.rolling(window=30).min(), **next(styles))[0])
    l2s.append(ax.semilogy(ih5data["steps"], ih5data["l2norms"], **next(styles))[0])
    l3s.append(tax.plot(ilogdata.index, ilogdata["time elapsed"], **next(styles))[0])

labels = [f"{ngpus} GPUs" for ngpus in gpus]
lgds = [
    ax.legend(l1s, labels, title="Loss", loc="upper left", bbox_to_anchor=(1.07, 1.025)),
    ax.legend(l2s, labels, title=r"$L_2$ err. of $u$", loc="center left", bbox_to_anchor=(1.07, 0.5)),
    ax.legend(l3s, labels, title="Run time", loc="lower left", bbox_to_anchor=(1.07, -0.025))
]
ax.add_artist(lgds[0])
ax.add_artist(lgds[1])
ax.add_artist(lgds[2])

# save
figdir.joinpath("scaling-tests").mkdir(parents=True, exist_ok=True)
fig.savefig(figdir.joinpath("scaling-tests", "nl2-nn32-npts8192-scaling.png"), bbox_inches="tight")

# ===============================================================
fig = pyplot.figure(figsize=(8, 6.5))
fig.suptitle(r"Comparing $(N_l, N_n, N_{bs}) \times N_{gpu}$ and $(N_l, N_n, N_{bs} \times N_{gpu})$")
gs = fig.add_gridspec(2, 4)

locs = [(0, slice(None, 2)), (0, slice(2, None)), (1, slice(None, 2))]
for iloc, ngpus, ilogdata, icompdata, ih5data, icomph5data in zip(locs, gpus[1:], logdata[1:], compdata[1:], h5data[1:], comph5data[1:]):
    styles = styles_def()
    ax = fig.add_subplot(gs[iloc])
    ax.set_title(rf"$(2, 32, 8192) \times {ngpus}$ v.s. $(2, 32, {8192*ngpus})$")
    ax.set_xlabel("Iteration")
    ax.set_ylabel(r"Aggregated loss and $L_2$ error of $u$")
    tax = ax.twinx()
    tax.set_ylabel("Run time (hours)")
    l1, = ax.semilogy(ilogdata.index, ilogdata.loss.rolling(window=30).min(), **next(styles))
    l2, = ax.semilogy(icompdata.index, icompdata.loss.rolling(window=30).min(), **next(styles))
    l3, = ax.semilogy(ih5data["steps"], ih5data["l2norms"], **next(styles))
    l4, = ax.semilogy(icomph5data["steps"], icomph5data["l2norms"], **next(styles))
    l5, = tax.plot(ilogdata.index, ilogdata["time elapsed"], **next(styles))
    l6, = tax.plot(icompdata.index, icompdata["time elapsed"], **next(styles))

# legend
labels = [r"$(N_l, N_n, N_{bs})\times N_{gpu}$", r"$(N_l, N_n, N_{bs}\times N_{gpu})$"]
fig.legend([l1, l2], labels, title="Loss", loc="upper left", bbox_to_anchor=(0.6, 0.44), labelspacing=0.)
fig.legend([l3, l4], labels, title=r"$L_2$ err. of $u$", loc="upper left", bbox_to_anchor=(0.6, 0.33), labelspacing=0.)
fig.legend([l5, l5], labels, title="Run time", loc="upper left", bbox_to_anchor=(0.6, 0.213), labelspacing=0.)

# save
figdir.joinpath("scaling-tests").mkdir(parents=True, exist_ok=True)
fig.savefig(figdir.joinpath("scaling-tests", "nl2-nn32-npts8192-multi-singl-gpus.png"))

In [ ]:
# strong scaling for nl3-nn128-npts65536
gpus = [1, 2, 4, 8]
logdata = []
h5data = []

for ngpus in gpus:

    bname = "exp-sum-scaling"
    cname = f"nl3-nn128-npts{65536//ngpus}-ngpus{ngpus}"
    logdata.append(log_parser(workdir.joinpath(bname, cname)))

    with h5open(workdir.joinpath("outputs", bname, f"{cname}-raw.h5"), "r") as h5file:
        h5data.append({
            "steps": h5file["walltime/steps"][...],
            "elapsedtimes": h5file["walltime/elapsedtimes"][...],
            "l2norms": h5file["walltime/l2norms/u/40.0"][...]
        })

styles_def = (
    cycler("ls", ["solid", "dotted", "dashed", "dashdot"]*5) + 
    cycler("color", pyplot.get_cmap("tab10").colors*2) + 
    cycler("lw", [2.0]*20) +
    cycler("alpha", [0.75]*20)
)

styles = styles_def()

# ===============================================================
fig, ax = pyplot.subplots(1, 1, figsize=(8, 4))
fig.suptitle("Aggregated loss and elapsed time v.s. iteration")

ax.set_xlabel("Iteration")
ax.set_ylabel(r"Aggregated loss and $L_2$ error of $u$")
ax.grid()
tax = ax.twinx()
tax.set_ylabel("Run time (hours)")

l1s, l2s, l3s = [], [], []
for ilogdata, ih5data in zip(logdata, h5data):
    l1s.append(ax.semilogy(ilogdata.index, ilogdata.loss.rolling(window=30).min(), **next(styles))[0])
    l2s.append(ax.semilogy(ih5data["steps"], ih5data["l2norms"], **next(styles))[0])
    l3s.append(tax.plot(ilogdata.index, ilogdata["time elapsed"], **next(styles))[0])

labels = [f"{ngpus} GPUs" for ngpus in gpus]
lgds = [
    ax.legend(l1s, labels, title="Loss", loc="upper left", bbox_to_anchor=(1.07, 1.025)),
    ax.legend(l2s, labels, title=r"$L_2$ err. of $u$", loc="center left", bbox_to_anchor=(1.07, 0.5)),
    ax.legend(l3s, labels, title="Run time", loc="lower left", bbox_to_anchor=(1.07, -0.025))
]
ax.add_artist(lgds[0])
ax.add_artist(lgds[1])
ax.add_artist(lgds[2])

# save
figdir.joinpath("scaling-tests").mkdir(parents=True, exist_ok=True)
fig.savefig(figdir.joinpath("scaling-tests", "nl3-nn128-npts65536-strong-scaling.png"), bbox_inches="tight")

In [ ]:
# strong scaling for nl2-nn32-npts65536
gpus = [1, 2, 4, 8]
logdata = []
h5data = []

for ngpus in gpus:

    bname = "exp-sum-scaling"
    cname = f"nl2-nn32-npts{65536//ngpus}-ngpus{ngpus}"
    logdata.append(log_parser(workdir.joinpath(bname, cname)))

    with h5open(workdir.joinpath("outputs", bname, f"{cname}-raw.h5"), "r") as h5file:
        h5data.append({
            "steps": h5file["walltime/steps"][...],
            "elapsedtimes": h5file["walltime/elapsedtimes"][...],
            "l2norms": h5file["walltime/l2norms/u/40.0"][...]
        })

styles_def = (
    cycler("ls", ["solid", "dotted", "dashed", "dashdot"]*5) + 
    cycler("color", pyplot.get_cmap("tab10").colors*2) + 
    cycler("lw", [2.0]*20) +
    cycler("alpha", [0.75]*20)
)

styles = styles_def()

# ===============================================================
fig, ax = pyplot.subplots(1, 1, figsize=(8, 4))
fig.suptitle("Aggregated loss and elapsed time v.s. iteration")

ax.set_xlabel("Iteration")
ax.set_ylabel(r"Aggregated loss and $L_2$ error of $u$")
ax.grid()
tax = ax.twinx()
tax.set_ylabel("Run time (hours)")

l1s, l2s, l3s = [], [], []
for ilogdata, ih5data in zip(logdata, h5data):
    l1s.append(ax.semilogy(ilogdata.index, ilogdata.loss.rolling(window=30).min(), **next(styles))[0])
    l2s.append(ax.semilogy(ih5data["steps"], ih5data["l2norms"], **next(styles))[0])
    l3s.append(tax.plot(ilogdata.index, ilogdata["time elapsed"], **next(styles))[0])

labels = [f"{ngpus} GPUs" for ngpus in gpus]
lgds = [
    ax.legend(l1s, labels, title="Loss", loc="upper left", bbox_to_anchor=(1.07, 1.025)),
    ax.legend(l2s, labels, title=r"$L_2$ err. of $u$", loc="center left", bbox_to_anchor=(1.07, 0.5)),
    ax.legend(l3s, labels, title="Run time", loc="lower left", bbox_to_anchor=(1.07, -0.025))
]
ax.add_artist(lgds[0])
ax.add_artist(lgds[1])
ax.add_artist(lgds[2])

# save
figdir.joinpath("scaling-tests").mkdir(parents=True, exist_ok=True)
fig.savefig(figdir.joinpath("scaling-tests", "nl2-nn32-npts65536-strong-scaling.png"), bbox_inches="tight")

In [ ]:
# weak scaling table
gpus = [1, 2, 4, 8]
cases = [(2, 32, 8192), (3, 128, 8192)]
indices = ["Time cost", "Efficiency", "Loss", r"$L_2$ err., $u$", r"$L_2$ err., $v$"]
columns = pandas.MultiIndex.from_product([cases, gpus])
columns = columns.set_names("GPUs", level=1)
data = pandas.DataFrame(data=None, index=indices, columns=columns)

bname = "exp-sum-scaling"
for (nl, nn, nbs), ngpus in itertools.product(cases, gpus):
    cname = f"nl{nl}-nn{nn}-npts{nbs}-ngpus{ngpus}"
    with h5open(workdir.joinpath("outputs", bname, f"{cname}-raw.h5"), "r") as h5file:
        data.loc["Time cost", ((nl, nn, nbs), ngpus)] = h5file["walltime/elapsedtimes"][-1]
        data.loc[r"$L_2$ err., $u$", ((nl, nn, nbs), ngpus)] = float(h5file["sterrs/u/l2norm"][...])
        data.loc[r"$L_2$ err., $v$", ((nl, nn, nbs), ngpus)] = float(h5file["sterrs/v/l2norm"][...])
    data.loc["Loss", ((nl, nn, nbs), ngpus)] = log_parser(workdir.joinpath(bname, cname)).loc[400000, "loss"]
    data.loc["Efficiency", ((nl, nn, nbs), ngpus)] = \
        100 * data.loc["Time cost", ((nl, nn, nbs), 1)].values[0] / \
            data.loc["Time cost", ((nl, nn, nbs), ngpus)].values[0]

out = data.style
out = out.format(formatter="{:5.2f}", subset=pandas.IndexSlice["Time cost", :])
out = out.format(formatter="{:2.0f}", subset=pandas.IndexSlice["Efficiency", :])
out = out.format(formatter="{:.1e}", subset=pandas.IndexSlice["Loss", :])
out = out.format(formatter="{:.1e}", subset=pandas.IndexSlice[r"$L_2$ err., $u$", :])
out = out.format(formatter="{:.1e}", subset=pandas.IndexSlice[r"$L_2$ err., $v$", :])

out = out.to_latex(
    column_format="lcccccccc",
    position="H",
    position_float="centering",
    hrules=True,
    label="table:weak-scaling-perf",
    caption=(
        "\n    Weak scaling performance for $(N_l, N_n, N_{bs})$ $=$ $(2, 32, 65536)$ and $(3, 128, 65536)$.%\n"
        "    Time costs denote the wall time required to finish 400k training iterations in hours.%\n"
        "    Efficiency here stands for weak scaling efficiency in $\%$.%\n"
        "    The aggregated losses were those at the last iteration.%\n"
        "    The $L_2$ errors were the overall spatial-temporal errors at the last training iteration.%\n",
        "\n    Weak scaling performance for $(N_l, N_n, N_{bs})=(2, 32, 65536)$ and $(3, 128, 65536)$\n"
    ),
    multicol_align="c",
)

patn = r"\\multicolumn(?:\{.*?\}){3}"
patn = rf"(^\s*?&\s*?{patn}\s*&\s*{patn}.*?$)"
repl = r"\g<1>\n\\cmidrule(rl){2-5} \\cmidrule(rl){6-9}"
out = re.sub(patn, repl, out, flags=re.MULTILINE)

out = re.sub(r"^(\\centering)$", r"\g<1>\n\\singlespacing", out, flags=re.MULTILINE)
out = re.sub(r"GPUs", "\\\\multicolumn{1}{r}{GPUs}", out)
out = re.sub(r"(^Time cost.*?)$", r"\g<1>\n\\addlinespace", out, flags=re.MULTILINE)
out = re.sub(r"(^Efficiency.*?)$", r"\g<1>\n\\addlinespace", out, flags=re.MULTILINE)
out = re.sub(r"(^Loss.*?)$", r"\g<1>\n\\addlinespace", out, flags=re.MULTILINE)
out = re.sub(r"(^\$L_2\$ err\., \$u\$.*?)$", r"\g<1>\n\\addlinespace", out, flags=re.MULTILINE)

print(out)

workdir.joinpath("tables", "scaling-tests").mkdir(parents=True, exist_ok=True)
with open(workdir.joinpath("tables", "scaling-tests", "weak-scaling.tex"), "w") as fobj:
    fobj.write(out)

In [ ]:
# strong scaling table
gpus = [1, 2, 4, 8]
cases = [(2, 32, 65536), (3, 128, 65536)]
indices = ["Time cost", "Efficiency", "Loss", r"$L_2$ err., $u$", r"$L_2$ err., $v$"]
columns = pandas.MultiIndex.from_product([cases, gpus])
columns = columns.set_names("GPUs", level=1)
data = pandas.DataFrame(data=None, index=indices, columns=columns)

bname = "exp-sum-scaling"
for (nl, nn, nbs), ngpus in itertools.product(cases, gpus):
    cname = f"nl{nl}-nn{nn}-npts{nbs//ngpus}-ngpus{ngpus}"
    with h5open(workdir.joinpath("outputs", bname, f"{cname}-raw.h5"), "r") as h5file:
        data.loc["Time cost", ((nl, nn, nbs), ngpus)] = h5file["walltime/elapsedtimes"][-1]
        data.loc[r"$L_2$ err., $u$", ((nl, nn, nbs), ngpus)] = float(h5file["sterrs/u/l2norm"][...])
        data.loc[r"$L_2$ err., $v$", ((nl, nn, nbs), ngpus)] = float(h5file["sterrs/v/l2norm"][...])
    data.loc["Loss", ((nl, nn, nbs), ngpus)] = log_parser(workdir.joinpath(bname, cname)).loc[400000, "loss"]
    data.loc["Efficiency", ((nl, nn, nbs), ngpus)] = \
        100 * data.loc["Time cost", ((nl, nn, nbs), 1)].values[0] / \
            data.loc["Time cost", ((nl, nn, nbs), ngpus)].values[0]

out = data.style
out = out.format(formatter="{:5.2f}", subset=pandas.IndexSlice["Time cost", :])
out = out.format(formatter="{:2.0f}", subset=pandas.IndexSlice["Efficiency", :])
out = out.format(formatter="{:.1e}", subset=pandas.IndexSlice["Loss", :])
out = out.format(formatter="{:.1e}", subset=pandas.IndexSlice[r"$L_2$ err., $u$", :])
out = out.format(formatter="{:.1e}", subset=pandas.IndexSlice[r"$L_2$ err., $v$", :])

out = out.to_latex(
    column_format="lcccccccc",
    position="H",
    position_float="centering",
    hrules=True,
    label="table:strong-scaling-perf",
    caption=(
        "\n    Strong scaling performance for $(N_l, N_n, N_{bs})$ $=$ $(2, 32, 65536)$ and $(3, 128, 65536)$.%\n"
        "    Time costs denote the wall time required to finish 400k training iterations in hours.%\n"
        "    Efficiency here stands for strong scaling efficiency in $\%$.%\n"
        "    The aggregated losses were those at the last iteration.%\n"
        "    The $L_2$ errors were the overall spatial-temporal errors at the last training iteration.%\n",
        "\n    Strong scaling performance for $(N_l, N_n, N_{bs})=(2, 32, 65536)$ and $(3, 128, 65536)$\n"
    ),
    multicol_align="c",
)

patn = r"\\multicolumn(?:\{.*?\}){3}"
patn = rf"(^\s*?&\s*?{patn}\s*&\s*{patn}.*?$)"
repl = r"\g<1>\n\\cmidrule(rl){2-5} \\cmidrule(rl){6-9}"
out = re.sub(patn, repl, out, flags=re.MULTILINE)

out = re.sub(r"^(\\centering)$", r"\g<1>\n\\singlespacing", out, flags=re.MULTILINE)
out = re.sub(r"GPUs", "\\\\multicolumn{1}{r}{GPUs}", out)
out = re.sub(r"(^Time cost.*?)$", r"\g<1>\n\\addlinespace", out, flags=re.MULTILINE)
out = re.sub(r"(^Efficiency.*?)$", r"\g<1>\n\\addlinespace", out, flags=re.MULTILINE)
out = re.sub(r"(^Loss.*?)$", r"\g<1>\n\\addlinespace", out, flags=re.MULTILINE)
out = re.sub(r"(^\$L_2\$ err\., \$u\$.*?)$", r"\g<1>\n\\addlinespace", out, flags=re.MULTILINE)

print(out)

workdir.joinpath("tables", "scaling-tests").mkdir(parents=True, exist_ok=True)
with open(workdir.joinpath("tables", "scaling-tests", "weak-scaling.tex"), "w") as fobj:
    fobj.write(out)